In [2]:
import pandas as pd
import math

# --- 설정 ---
# 읽을 CSV 파일 이름
csv_file_name = "../data_csv/11_10/강원도_모범음식점.csv"

# 생성할 SQL 파일 이름
sql_file_name = "insert_kangwondo_businesses.sql"

# 생성할 테이블 이름 (따옴표 없음)
table_name = "kangwondo_businesses"
# ----------------

def clean_value_mysql(value):
    """
    Python 데이터를 SQL 문자열로 안전하게 변환합니다.
    - None 또는 NaN -> 'NULL'
    - 문자열 -> 작은따옴표로 감싸고, 내부의 작은따옴표 이스케이프
    """
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        # MySQL은 '를 ''로, \를 \\로 이스케이프합니다.
        cleaned_value = value.replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"
    else:
        # 숫자나 다른 타입은 문자열로 변환
        return str(value)

# 1. CSV 파일 읽기 (한글 인코딩 처리)
try:
    df = pd.read_csv(csv_file_name, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_name, encoding='utf-8-sig')
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame() # 오류 시 빈 데이터프레임

if not df.empty:
    print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
    
    # 2. CREATE TABLE 쿼리 생성
    # 컬럼명 리스트 (따옴표 없음)
    columns = [col for col in df.columns]
    column_sql = ", ".join(columns)
    
    create_table_sql = f"""
-- 강원도 업종 테이블 생성 (MySQL용, 따옴표 없음)
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    업종 VARCHAR(255) CHARACTER SET utf8mb4,
    업소명 VARCHAR(255) CHARACTER SET utf8mb4,
    주소 VARCHAR(255) CHARACTER SET utf8mb4,
    연락처 VARCHAR(50)
);
\n"""

    # 3. SQL 파일 쓰기 시작
    try:
        with open(sql_file_name, 'w', encoding='utf-8') as f:
            # 파일 상단에 데이터베이스 선택 주석 추가
            f.write("-- 사용할 데이터베이스를 선택하세요 (예: USE your_database_name;)\n")
            
            # CREATE TABLE 문 쓰기
            f.write(create_table_sql)
            f.write("\n")
            
            values_batch = []
            
            # INSERT 문 시작 부분 (따옴표 없음)
            insert_prefix = f"INSERT INTO {table_name} ({column_sql}) VALUES \n"
            
            # 4. DataFrame을 행별로 순회하며 VALUES 생성
            for row in df.itertuples(index=False):
                cleaned_row = [
                    clean_value_mysql(getattr(row, '업종')),
                    clean_value_mysql(getattr(row, '업소명')),
                    clean_value_mysql(getattr(row, '주소')),
                    clean_value_mysql(getattr(row, '연락처'))
                ]
                values_batch.append(f"({', '.join(cleaned_row)})")
            
            # 5. INSERT 쿼리 파일에 쓰기
            if values_batch:
                # 모든 데이터를 하나의 INSERT 문으로 합쳐서 씀
                f.write(insert_prefix + ",\n".join(values_batch) + ";\n\n")
                
        print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
        print(f"총 {len(df)}개의 행이 파일에 저장되었습니다.")

    except Exception as e:
        print(f"SQL 파일 생성 중 오류: {e}")

'../data_csv/11_10/강원도_모범음식점.csv' 파일 읽기 성공. 988개 행 발견.
SQL 파일 생성 중 오류: 'Pandas' object has no attribute '업종'


In [1]:
import os

print(os.listdir("../data_csv/11_10"))

['춘천시_버스정류장노선정보_20250212.csv', '춘천시_요식업소현황_2504.csv', '강원도_모범음식점.csv', '춘천시_요식업소_좌표포함.csv']


In [3]:
import pandas as pd

df = pd.read_csv("../data_csv/11_10/강원도_모범음식점.csv", encoding="cp949")
print(df.columns)

Index(['연번', '시도', '시군구', '업태', '주메뉴', '업소명', '주소', '최초 지정일자'], dtype='object')


In [10]:
import pandas as pd
import math

# --- 설정 ---
# 읽을 CSV 파일 이름
csv_file_name = "../data_csv/11_10/강원도_모범음식점.csv"

# 생성할 SQL 파일 이름
sql_file_name = "insert_kangwondo_businesses.sql"

# 생성할 테이블 이름 (따옴표 없음)
table_name = "kangwondo_businesses"
# ----------------

def clean_value_mysql(value):
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        cleaned_value = value.replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"
    else:
        return str(value)

# 1. CSV 파일 읽기 (한글 인코딩 처리)
try:
    df = pd.read_csv(csv_file_name, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_name, encoding='utf-8-sig')
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame()

if not df.empty:
    print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
    
    # CSV 실제 컬럼
    # ['연번','시도','시군구','업태','주메뉴','업소명','주소','최초 지정일자']
    # 여기서 '최초 지정일자'는 제외
    real_columns = ["연번", "시도", "시군구", "업태", "주메뉴", "업소명", "주소"]

    column_sql = ", ".join(real_columns)

    # CREATE TABLE 문 생성 (너 스타일 유지)
    create_table_sql = f"""
-- 강원도 업종 테이블 생성 (MySQL용, 따옴표 없음)
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    연번 INT,
    시도 VARCHAR(50) CHARACTER SET utf8mb4,
    시군구 VARCHAR(50) CHARACTER SET utf8mb4,
    업태 VARCHAR(255) CHARACTER SET utf8mb4,
    주메뉴 VARCHAR(255) CHARACTER SET utf8mb4,
    업소명 VARCHAR(255) CHARACTER SET utf8mb4,
    주소 VARCHAR(255) CHARACTER SET utf8mb4
);
\n"""

    # SQL 파일 생성
    try:
        with open(sql_file_name, 'w', encoding='utf-8') as f:

            f.write("-- 사용할 데이터베이스를 선택하세요 (예: USE your_database_name;)\n")
            f.write(create_table_sql)
            f.write("\n")

            insert_prefix = f"INSERT INTO {table_name} ({column_sql}) VALUES \n"

            values_batch = []

            # CSV → INSERT VALUES 변환 (최초 지정일자 제외)
            for row in df.itertuples(index=False):
                cleaned_row = [
                    clean_value_mysql(getattr(row, '연번')),
                    clean_value_mysql(getattr(row, '시도')),
                    clean_value_mysql(getattr(row, '시군구')),
                    clean_value_mysql(getattr(row, '업태')),
                    clean_value_mysql(getattr(row, '주메뉴')),
                    clean_value_mysql(getattr(row, '업소명')),
                    clean_value_mysql(getattr(row, '주소')),
                ]
                values_batch.append(f"({', '.join(cleaned_row)})")

            # INSERT 문 전체 한번에 작성
            if values_batch:
                f.write(insert_prefix + ",\n".join(values_batch) + ";\n\n")
                
        print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
        print(f"총 {len(df)}개의 행이 파일에 저장되었습니다.")

    except Exception as e:
        print(f"SQL 파일 생성 중 오류: {e}")

'../data_csv/11_10/강원도_모범음식점.csv' 파일 읽기 성공. 988개 행 발견.
'insert_kangwondo_businesses.sql' 파일 생성이 완료되었습니다.
총 988개의 행이 파일에 저장되었습니다.


In [11]:
import pandas as pd
import math

# --- 설정 ---
# 읽을 CSV 파일 이름
csv_file_name = "../data_csv/11_11/강원도업종.csv"

# 생성할 SQL 파일 이름
sql_file_name = "insert_kangwondo_businesses.sql"

# 생성할 테이블 이름 (따옴표 없음)
table_name = "kangwondo_businesses"
# ----------------

def clean_value_mysql(value):
    """
    Python 데이터를 SQL 문자열로 안전하게 변환합니다.
    - None 또는 NaN -> 'NULL'
    - 문자열 -> 작은따옴표로 감싸고, 내부의 작은따옴표 이스케이프
    """
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        cleaned_value = value.replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"
    else:
        return str(value)

# 1. CSV 파일 읽기 (한글 인코딩 처리)
try:
    df = pd.read_csv(csv_file_name, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_name, encoding='utf-8-sig')
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame() # 오류 시 빈 데이터프레임

if not df.empty:
    print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
    
    # 2. CREATE TABLE 쿼리 생성
    # CSV 컬럼명 그대로 사용 (따옴표 없음)
    columns = [col for col in df.columns]   # 예: ['업종', '업소명', '주소', '연락처']
    column_sql = ", ".join(columns)
    
    create_table_sql = f"""
-- 강원도 업종 테이블 생성 (MySQL용, 따옴표 없음)
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    업종 VARCHAR(255) CHARACTER SET utf8mb4,
    업소명 VARCHAR(255) CHARACTER SET utf8mb4,
    주소 VARCHAR(255) CHARACTER SET utf8mb4,
    연락처 VARCHAR(50)
);
\n"""

    # 3. SQL 파일 쓰기 시작
    try:
        with open(sql_file_name, 'w', encoding='utf-8') as f:
            # 파일 상단에 데이터베이스 선택 주석 추가
            f.write("-- 사용할 데이터베이스를 선택하세요 (예: USE your_database_name;)\n")
            
            # CREATE TABLE 문 쓰기
            f.write(create_table_sql)
            f.write("\n")
            
            values_batch = []
            
            # INSERT 문 시작 부분 (따옴표 없음)
            insert_prefix = f"INSERT INTO {table_name} ({column_sql}) VALUES \n"
            
            # 4. DataFrame을 행별로 순회하며 VALUES 생성
            for row in df.itertuples(index=False):
                cleaned_row = [
                    clean_value_mysql(getattr(row, '업종')),
                    clean_value_mysql(getattr(row, '업소명')),
                    clean_value_mysql(getattr(row, '주소')),
                    clean_value_mysql(getattr(row, '연락처'))
                ]
                values_batch.append(f"({', '.join(cleaned_row)})")
            
            # 5. INSERT 쿼리 파일에 쓰기
            if values_batch:
                f.write(insert_prefix + ",\n".join(values_batch) + ";\n\n")
                
        print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
        print(f"총 {len(df)}개의 행이 파일에 저장되었습니다.")

    except Exception as e:
        print(f"SQL 파일 생성 중 오류: {e}")
else:
    print("데이터프레임이 비어 있습니다. CSV 경로 또는 인코딩을 확인하세요.")

'../data_csv/11_11/강원도업종.csv' 파일 읽기 성공. 356개 행 발견.
'insert_kangwondo_businesses.sql' 파일 생성이 완료되었습니다.
총 356개의 행이 파일에 저장되었습니다.


In [12]:
import os
print(os.getcwd())
print(os.listdir())

/workspaces/company_data/first_week/analyzing_data_csv
['11_10_숙박업분석.ipynb', '11_10_강원도버스노선.ipynb', '11_12_업종이랑 버스정류장 merge.ipynb', '11_13_춘천시_버스노선.ipynb', '11_10_소재지변경.ipynb', '11_13_강원도모범음식점1.ipynb', '11_11_춘천시업종버스.ipynb', '11_11_강원업종.ipynb', '11_10_모범음식점.ipynb', 'insert_kangwondo_businesses.sql', '11_10_춘천시_요식업소.ipynb', '11_11_강원도주민등록인구및세대.ipynb', '11_10_강원도모범음식점.ipynb']


In [13]:
import pandas as pd

# 파일 이름
csv_file_name = "../data_csv/11_10/강원도_모범음식점.csv"
sql_file_name = "insert_kangwondo_businesses.sql"
table_name = "kangwondo_businesses"

def clean_value_mysql(value):
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        return "'" + value.replace("\\", "\\\\").replace("'", "''") + "'"
    else:
        return str(value)

# CSV 읽기
try:
    df = pd.read_csv(csv_file_name, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_name, encoding='utf-8-sig')

print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
print("컬럼명:", list(df.columns))

# 실제 CSV 컬럼 그대로 사용
columns = ['연번', '시도', '시군구', '업태', '주메뉴', '업소명', '주소', '최초 지정일자']
column_sql = ", ".join(columns)

# CREATE TABLE
create_table_sql = f"""
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    연번 INT,
    시도 VARCHAR(50),
    시군구 VARCHAR(50),
    업태 VARCHAR(100),
    주메뉴 VARCHAR(255),
    업소명 VARCHAR(255),
    주소 VARCHAR(255),
    최초_지정일자 VARCHAR(50)
);
"""

with open(sql_file_name, 'w', encoding='utf-8') as f:
    f.write("-- 사용할 데이터베이스 선택: USE your_database;\n\n")
    f.write(create_table_sql + "\n")

    f.write(f"INSERT INTO {table_name} ({column_sql}) VALUES\n")

    values = []
    for row in df.itertuples(index=False):
        values.append("(" + ", ".join([
            clean_value_mysql(getattr(row, '연번')),
            clean_value_mysql(getattr(row, '시도')),
            clean_value_mysql(getattr(row, '시군구')),
            clean_value_mysql(getattr(row, '업태')),
            clean_value_mysql(getattr(row, '주메뉴')),
            clean_value_mysql(getattr(row, '업소명')),
            clean_value_mysql(getattr(row, '주소')),
            clean_value_mysql(getattr(row, '최초 지정일자'))
        ]) + ")")

    f.write(",\n".join(values) + ";\n")

print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
print(f"총 {len(df)}개의 행이 파일에 저장되었습니다.")

'../data_csv/11_10/강원도_모범음식점.csv' 파일 읽기 성공. 988개 행 발견.
컬럼명: ['연번', '시도', '시군구', '업태', '주메뉴', '업소명', '주소', '최초 지정일자']


AttributeError: 'Pandas' object has no attribute '최초 지정일자'